# STRPy: Comparison Study

This notebook demonstrates how to compare different decomposition methods using synthetic data, similar to the simulation study in Dokumentov & Hyndman (2022).

## Overview

We'll:
1. Generate synthetic data with known components
2. Apply different decomposition methods
3. Compare accuracy using RMSE
4. Visualize differences

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from strpy import generate_synthetic_data
from strpy.simulations import rmse

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(2020)

## 1. Generate Test Data

We'll generate data similar to the paper's simulation study.

In [ ]:
# Test different noise levels
gamma_values = [0.2, 0.4, 0.6]
n_simulations = 20  # Reduced from 100 for demo
n = 365 * 3 + 1  # ~3 years

results = []

for data_type in ['stochastic', 'deterministic']:
    for gamma in gamma_values:
        print(f"\nGenerating {data_type} data with gamma={gamma}...")
        
        for i in range(n_simulations):
            df = generate_synthetic_data(
                n=n,
                periods=(7, 365),
                alpha=1.0,
                beta=1.0,
                gamma=gamma,
                data_type=data_type,
                random_seed=1000 + i
            )
            
            results.append({
                'type': data_type,
                'gamma': gamma,
                'sim': i,
                'data': df
            })

print(f"\nGenerated {len(results)} datasets")

## 2. Visualize Sample Data

In [ ]:
# Plot examples from each configuration
fig, axes = plt.subplots(3, 2, figsize=(14, 10))

for i, gamma in enumerate(gamma_values):
    # Stochastic
    df_stoch = [r['data'] for r in results 
                if r['type'] == 'stochastic' and r['gamma'] == gamma][0]
    axes[i, 0].plot(df_stoch['data'], 'b-', linewidth=0.8, alpha=0.8)
    axes[i, 0].set_ylabel('Value', fontsize=10)
    axes[i, 0].set_title(f'Stochastic, γ={gamma}', fontsize=11)
    axes[i, 0].grid(True, alpha=0.3)
    
    # Deterministic
    df_det = [r['data'] for r in results 
              if r['type'] == 'deterministic' and r['gamma'] == gamma][0]
    axes[i, 1].plot(df_det['data'], 'r-', linewidth=0.8, alpha=0.8)
    axes[i, 1].set_ylabel('Value', fontsize=10)
    axes[i, 1].set_title(f'Deterministic, γ={gamma}', fontsize=11)
    axes[i, 1].grid(True, alpha=0.3)

axes[2, 0].set_xlabel('Time (days)', fontsize=10)
axes[2, 1].set_xlabel('Time (days)', fontsize=10)

plt.tight_layout()
plt.show()

## 3. Statistical Summary

Let's examine the properties of the generated data.

In [ ]:
# Calculate signal-to-noise ratios
snr_summary = []

for result in results:
    df = result['data']
    signal = df['trend'] + df['seasonal_1'] + df['seasonal_2']
    noise = df['remainder']
    
    snr = signal.std() / noise.std()
    
    snr_summary.append({
        'Type': result['type'],
        'Gamma': result['gamma'],
        'SNR': snr
    })

snr_df = pd.DataFrame(snr_summary)
snr_grouped = snr_df.groupby(['Type', 'Gamma'])['SNR'].agg(['mean', 'std'])

print("\nSignal-to-Noise Ratio Statistics:")
print("="*50)
print(snr_grouped)
print("="*50)

## 4. Component Analysis

Analyze the variance contribution of each component.

In [ ]:
# Calculate average variance contributions
variance_analysis = []

for result in results:
    df = result['data']
    
    var_trend = df['trend'].var()
    var_weekly = df['seasonal_1'].var()
    var_yearly = df['seasonal_2'].var()
    var_remainder = df['remainder'].var()
    total_var = var_trend + var_weekly + var_yearly + var_remainder
    
    variance_analysis.append({
        'Type': result['type'],
        'Gamma': result['gamma'],
        'Trend %': 100 * var_trend / total_var,
        'Weekly %': 100 * var_weekly / total_var,
        'Yearly %': 100 * var_yearly / total_var,
        'Remainder %': 100 * var_remainder / total_var,
    })

var_df = pd.DataFrame(variance_analysis)
var_grouped = var_df.groupby(['Type', 'Gamma']).mean()

print("\nAverage Variance Contribution (%):")
print("="*70)
print(var_grouped)
print("="*70)

In [ ]:
# Visualize variance contributions
fig, axes = plt.subplots(2, 3, figsize=(15, 8))
axes = axes.flatten()

idx = 0
for data_type in ['stochastic', 'deterministic']:
    for gamma in gamma_values:
        subset = var_df[(var_df['Type'] == data_type) & (var_df['Gamma'] == gamma)]
        
        means = subset[['Trend %', 'Weekly %', 'Yearly %', 'Remainder %']].mean()
        
        axes[idx].bar(range(4), means.values, 
                     color=['red', 'green', 'blue', 'gray'], alpha=0.7)
        axes[idx].set_xticks(range(4))
        axes[idx].set_xticklabels(['Trend', 'Weekly', 'Yearly', 'Rem'], rotation=45)
        axes[idx].set_ylabel('Variance %')
        axes[idx].set_title(f'{data_type.title()}, γ={gamma}')
        axes[idx].grid(True, alpha=0.3, axis='y')
        
        idx += 1

plt.tight_layout()
plt.show()

## 5. Baseline Comparison: Naive Decomposition

As a baseline, let's try a simple moving average decomposition.

In [ ]:
def simple_ma_decomposition(data, period=7):
    """
    Simple moving average decomposition (baseline).
    """
    # Trend: centered moving average
    trend = pd.Series(data).rolling(window=period, center=True).mean().values
    
    # Detrend
    detrended = data - np.nan_to_num(trend, nan=0)
    
    # Seasonal: average by position in cycle
    n = len(data)
    seasonal = np.zeros(n)
    for i in range(period):
        indices = np.arange(i, n, period)
        seasonal[indices] = np.nanmean(detrended[indices])
    
    # Remainder
    remainder = data - np.nan_to_num(trend, nan=0) - seasonal
    
    return {
        'trend': np.nan_to_num(trend, nan=0),
        'seasonal': seasonal,
        'remainder': remainder
    }

# Test on one series
test_df = results[0]['data']
simple_result = simple_ma_decomposition(test_df['data'].values, period=7)

# Plot
fig, axes = plt.subplots(4, 1, figsize=(14, 10))

axes[0].plot(test_df['data'], 'k-', linewidth=0.8, label='Data')
axes[0].set_title('Simple Moving Average Decomposition (Baseline)', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(test_df['trend'], 'r--', linewidth=1.5, alpha=0.5, label='True')
axes[1].plot(simple_result['trend'], 'r-', linewidth=1, label='Estimated')
axes[1].set_ylabel('Trend')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

axes[2].plot(test_df['seasonal_1'][:100], 'g--', linewidth=1.5, alpha=0.5, label='True')
axes[2].plot(simple_result['seasonal'][:100], 'g-', linewidth=1, label='Estimated')
axes[2].set_ylabel('Seasonal')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

axes[3].plot(simple_result['remainder'], 'gray', linewidth=0.5, alpha=0.7)
axes[3].set_ylabel('Remainder')
axes[3].set_xlabel('Time (days)')
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate errors
trend_rmse = rmse(test_df['trend'].values - simple_result['trend'])
seasonal_rmse = rmse(test_df['seasonal_1'].values - simple_result['seasonal'])

print(f"\nBaseline (Simple MA) RMSE:")
print(f"  Trend:    {trend_rmse:.4f}")
print(f"  Seasonal: {seasonal_rmse:.4f}")

## 6. Monte Carlo Simulation Summary

Summarize results across all simulations.

In [ ]:
# Apply simple MA to all datasets and collect errors
baseline_errors = []

print("Running baseline decomposition on all datasets...")
for i, result in enumerate(results):
    if i % 20 == 0:
        print(f"  Progress: {i}/{len(results)}")
    
    df = result['data']
    decomp = simple_ma_decomposition(df['data'].values, period=7)
    
    baseline_errors.append({
        'Type': result['type'],
        'Gamma': result['gamma'],
        'Trend RMSE': rmse(df['trend'].values - decomp['trend']),
        'Seasonal RMSE': rmse(df['seasonal_1'].values - decomp['seasonal']),
    })

error_df = pd.DataFrame(baseline_errors)
error_summary = error_df.groupby(['Type', 'Gamma']).mean()

print("\n" + "="*60)
print("Baseline Method (Simple MA) - Average RMSE")
print("="*60)
print(error_summary)
print("="*60)
print(f"\nNote: Lower RMSE indicates better decomposition accuracy.")
print(f"These baseline results would be compared with STR, STL, and TBATS.")

## 7. Visualization of Error Distribution

In [ ]:
# Box plots of RMSE
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Trend RMSE
for data_type in ['stochastic', 'deterministic']:
    subset = error_df[error_df['Type'] == data_type]
    
    data_to_plot = [subset[subset['Gamma'] == g]['Trend RMSE'].values 
                    for g in gamma_values]
    
    positions = np.arange(len(gamma_values)) * 2 + (0 if data_type == 'stochastic' else 0.5)
    bp = axes[0].boxplot(data_to_plot, positions=positions, widths=0.4,
                         patch_artist=True,
                         boxprops=dict(facecolor='lightblue' if data_type == 'stochastic' else 'lightcoral'))

axes[0].set_xticks(np.arange(len(gamma_values)) * 2 + 0.25)
axes[0].set_xticklabels([f'γ={g}' for g in gamma_values])
axes[0].set_ylabel('RMSE')
axes[0].set_title('Trend RMSE Distribution')
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].legend([plt.Line2D([0], [0], color='lightblue', lw=4),
                plt.Line2D([0], [0], color='lightcoral', lw=4)],
               ['Stochastic', 'Deterministic'])

# Seasonal RMSE
for data_type in ['stochastic', 'deterministic']:
    subset = error_df[error_df['Type'] == data_type]
    
    data_to_plot = [subset[subset['Gamma'] == g]['Seasonal RMSE'].values 
                    for g in gamma_values]
    
    positions = np.arange(len(gamma_values)) * 2 + (0 if data_type == 'stochastic' else 0.5)
    bp = axes[1].boxplot(data_to_plot, positions=positions, widths=0.4,
                         patch_artist=True,
                         boxprops=dict(facecolor='lightgreen' if data_type == 'stochastic' else 'lightyellow'))

axes[1].set_xticks(np.arange(len(gamma_values)) * 2 + 0.25)
axes[1].set_xticklabels([f'γ={g}' for g in gamma_values])
axes[1].set_ylabel('RMSE')
axes[1].set_title('Seasonal RMSE Distribution')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].legend([plt.Line2D([0], [0], color='lightgreen', lw=4),
                plt.Line2D([0], [0], color='lightyellow', lw=4)],
               ['Stochastic', 'Deterministic'])

plt.tight_layout()
plt.show()

## 8. Conclusions

### Current Findings

From our baseline analysis:
- Simple moving average provides a basic decomposition
- Error increases with noise level (as expected)
- Deterministic and stochastic data show different characteristics

### Next Steps

When STR implementation is complete, this notebook will be extended to include:

1. **STR Decomposition** - Full implementation with regularization
2. **STL Comparison** - Using `statsmodels` or custom implementation
3. **Statistical Testing** - Paired t-tests for significance
4. **Performance Metrics**:
   - RMSE for each component
   - Computational time
   - Memory usage

5. **Visualization**:
   - Side-by-side component comparisons
   - Error heatmaps
   - Method rankings

### References

Dokumentov, A., & Hyndman, R. J. (2022). STR: Seasonal-Trend decomposition using Regression. *INFORMS Journal on Data Science*, 1(1), 50-62.

In [ ]:
# Save results for later use
# error_df.to_csv('../data/comparison_results.csv', index=False)
# print("Results saved!")